# a graph from a table

we'll use the extended chartability principles to experiment with a small graph.s

In [1]:
import networkx
from nbconvert_a11y.repr import get_table, TableOptions
shell.tangle.parser.fence_methods["graphviz"] = "graphviz:Source"

In [2]:
%%
create a graph in graphviz

    graph =\
```graphviz
graph LR {
    Compromising -- {Robust Understandable}
    Assistive -- {Perceivable Understandable}
    Flexible -- {Perceivable Operable Robust}
    Robust -- Perceivable -- Operable -- Robust [style=dashed]
    Perceivable -- Understandable -- Robust -- Perceivable [style=dashed]
}
```

    pour, caf = map(str.split, (
Percievable Operable Understandable Robust
Compromising Assistive Flexible

    ).splitlines())

then cast it to a networkx graph object 

    G = networkx.nx_pydot.read_dot(io.StringIO(graph.source))

nodes are indexes that have meaning in space.
there are multiple layouts

the positions would be hidden visually and non visually

In [3]:
chartability = await Series(["https://raw.githubusercontent.com/Chartability/POUR-CAF/main/README.md"]).http.get()

In [4]:
from nbconvert_a11y.table import new

In [5]:
%%
    rules = chartability.apply(shell.tangle.parser.parser.render).apply(bs4.BeautifulSoup, features="lxml")[0]
    
    hs = "h1,h2,h3,h4,h5,h6"
    headings = collections.defaultdict(list)
    for node in rules.select(F"{hs}, :is({hs}) ~ :not({hs})"):
        if node.name in hs.split(","):
            key = node
        else:
            headings[key].append(node)
    
    headings = Series(headings).to_frame("body")
    headings["level"] = headings.index.map(lambda x: int(x.name.lstrip("h"))).values
    
    headings = headings.reset_index().set_index(headings.index.attrgetter("string").rename("Heading"))
    headings["body"] = headings["body"].apply(lambda x: new("section", *x)).apply(lambda x: new("section", *x))
    headings["description"] = headings["body"].attrgetter("string")


In [6]:
%%
compute different layouts for our graph

    positions = Series({
        networkx.arf_layout: networkx.arf_layout(G),
        # networkx.bipartite_layout: networkx.bipartite_layout(G, G.nodes),
        networkx.circular_layout: networkx.circular_layout(G),
        networkx.spring_layout: networkx.spring_layout(G),
        networkx.kamada_kawai_layout: networkx.kamada_kawai_layout(G),
        networkx.planar_layout: networkx.planar_layout(G),
        networkx.shell_layout: networkx.shell_layout(G),
        networkx.spectral_layout: networkx.spectral_layout(G),
        networkx.spiral_layout: networkx.spiral_layout(G),
        networkx.spring_layout: networkx.spring_layout(G),
    }).series().T
    
    positions.columns = positions.columns.attrgetter("__name__").str.split("_").itemgetter(0)
    positions = positions.stack().series().rename(columns=dict(zip(range(2), "xy"))).unstack()
    
    positions = pandas.concat(dict(info=headings.loc[positions.index]), axis=1).join(
        positions
    ).drop(columns=[("info", "body"), ("info", "index"), ("info", "level")])
    
    positions["info", "heuristic"] = positions.index.map(set(pour).__contains__).map(
        ["wcag", "chartability"].__getitem__
    ).values

In [7]:
%%
make a dataframe of the information required to present the graph nodes and edges

    df = DataFrame(index=DataFrame(G.edges).drop(columns=2).drop_duplicates().set_index([0, 1]).index.rename(list("xy")))
    
    df = pandas.concat([
        positions.loc[df.index.get_level_values(0)].rename(columns={"x": "x0", "y": "y0"}, level=0).drop(("info",), axis=1).reset_index(drop=True),
        positions.loc[df.index.get_level_values(1)].rename(columns={"x": "x1", "y": "y1"}, level=0).drop(("info",), axis=1).reset_index(drop=True)
    ], axis=1).set_index(df.index)

ca885898-60e2-11ef-80ad-18cc18dc77e7.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
ca885898-60e2-11ef-80ad-18cc18dc77e7.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.


In [8]:
%%
show the whole thing

    section = df.pipe(get_table, id="edges", options=TableOptions(max_columns=1000))
    section.table.caption.clear()
    section.table.insert(0, new("caption", F"a graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges."))
    section.table.attrs["class"] = "visual"
    layouts = new(
        "fieldset",
        new("legend","layouts"),
        new("label", new("input", type="checkbox", checked="", name="active", onchange=
```javascript
document.getElementById(`edges`).classList.toggle("visual", this.checked);
                         console.log(this)
```
                        ), "active"),
        *(
            new("label",
                new("input", type="radio", name="layout", **{"aria-controls": F"nodes-layout-{layout}"}, onchange=
```javascript
document.querySelectorAll(`[name=layout]`).forEach((element) => {
    var target = document.getElementById(element.getAttribute(`aria-controls`));
    target.setAttribute(`media`, element.checked ? `all` : `none`);
})
```
               ), layout) for layout in positions["x"].columns
        )   
    )
    layouts.select_one("[name=layout]").attrs["checked"] = ""
    section.form.append(layouts)
    for i, layout in enumerate(positions["x"].columns):
        section.append(new("style",
```css
#edges.visual {
    tr{
        --x0: var(--x0-%s);
        --x0-min: var(--x0-%s-min);
        --x0-max: var(--x0-%s-max);
        --x0-diff: var(--x0-%s-diff);
        --x1: var(--x1-%s);
        --x1-min: var(--x0-%s-min);
        --x1-max: var(--x0-%s-max);
        --x1-diff: var(--x0-%s-diff);
        --y0: var(--y0-%s);
        --y0-min: var(--y0-%s-min);
        --y0-max: var(--y0-%s-max);
        --y0-diff: var(--y0-%s-diff);
        --y1: var(--y1-%s);
        --y1-min: var(--y0-%s-min);
        --y1-max: var(--y0-%s-max);
        --y1-diff: var(--y0-%s-diff);
    }
}
```
        .replace("%s", layout), id=F"nodes-layout-{layout}", media=i and "none" or "all"))

    section.form.append(layouts)
    HTML(section)

'<section aria-labelledby="edges-figcaption" id="edges-region"><figure><figcaption id="edges-figcaption"></figcaption><table aria-colcount="34" aria-labelledby="edges-figcaption edges-caption" aria-rowcount="17" class="visual" id="edges" role="table" style="--0-min: -1.0882; --0-max: 2.1980; --0-diff: 3.2862; --1-min: -0.9010; --1-max: 1.0000; --1-diff: 1.9010; --2-min: -0.8919; --2-max: 0.7909; --2-diff: 1.6828; --3-min: -1.0000; --3-max: 0.9068; --3-diff: 1.9068; --4-min: -1.0000; --4-max: 1.0000; --4-diff: 2.0000; --5-min: -1.0000; --5-max: 0.9010; --5-diff: 1.9010; --6-min: -0.7075; --6-max: 0.6468; --6-diff: 1.3544; --7-min: -0.3174; --7-max: 0.4555; --7-diff: 0.7729; --8-min: -0.0794; --8-max: 2.2014; --8-diff: 2.2808; --9-min: -0.9749; --9-max: 0.9749; --9-diff: 1.9499; --10-min: -1.0000; --10-max: 0.6273; --10-diff: 1.6273; --11-min: -0.9700; --11-max: 0.7587; --11-diff: 1.7286; --12-min: -0.4286; --12-max: 0.5714; --12-diff: 1.0000; --13-min: -0.9749; --13-max: 0.9749; --13-diff: 1.9499; --14-min: -1.0000; --14-max: 1.0000; --14-diff: 2.0000; --15-min: -0.8335; --15-max: 0.7373; --15-diff: 1.5708; --16-min: -1.0882; --16-max: 2.1980; --16-diff: 3.2862; --17-min: -0.9010; --17-max: 0.6235; --17-diff: 1.5245; --18-min: -0.6339; --18-max: 0.9467; --18-diff: 1.5805; --19-min: -1.0000; --19-max: 0.4405; --19-diff: 1.4405; --20-min: -0.8000; --20-max: 1.0000; --20-diff: 1.8000; --21-min: -0.6235; --21-max: 0.9010; --21-diff: 1.5245; --22-min: -0.7075; --22-max: 0.6468; --22-diff: 1.3544; --23-min: -1.0000; --23-max: 0.4555; --23-diff: 1.4555; --24-min: -1.1069; --24-max: 2.2014; --24-diff: 3.3083; --25-min: -0.9749; --25-max: 0.9749; --25-diff: 1.9499; --26-min: -1.0000; --26-max: 0.5305; --26-diff: 1.5305; --27-min: -0.9700; --27-max: 0.7536; --27-diff: 1.7236; --28-min: -0.4286; --28-max: 0.5714; --28-diff: 1.0000; --29-min: -0.9749; --29-max: 0.9749; --29-diff: 1.9499; --30-min: -0.1623; --30-max: 1.0000; --30-diff: 1.1623; --31-min: -0.7240; --31-max: 0.8201; --31-diff: 1.5441;"><caption>a graph with 7 nodes and 13 edges.</caption><caption id="edges-caption"></caption><thead role="rowgroup"><tr aria-selected="true" role="row" style=\'---0: &lt;th aria-colindex="1" aria-rowindex="1" role="colheader" rowspan="2" style=""&gt;x&lt;/th&gt;;  ---1: &lt;th aria-colindex="2" aria-rowindex="1" role="colheader" rowspan="2" style=""&gt;y&lt;/th&gt;;  ---2: &lt;th aria-colindex="3" aria-rowindex="1" role="colheader" style=""&gt;x0&lt;/th&gt;;  ---3: &lt;th aria-colindex="4" aria-rowindex="1" role="colheader" style=""&gt;x0&lt;/th&gt;;  ---4: &lt;th aria-colindex="5" aria-rowindex="1" role="colheader" style=""&gt;x0&lt;/th&gt;;  ---5: &lt;th aria-colindex="6" aria-rowindex="1" role="colheader" style=""&gt;x0&lt;/th&gt;;  ---6: &lt;th aria-colindex="7" aria-rowindex="1" role="colheader" style=""&gt;x0&lt;/th&gt;;  ---7: &lt;th aria-colindex="8" aria-rowindex="1" role="colheader" style=""&gt;x0&lt;/th&gt;;  ---8: &lt;th aria-colindex="9" aria-rowindex="1" role="colheader" style=""&gt;x0&lt;/th&gt;;  ---9: &lt;th aria-colindex="10" aria-rowindex="1" role="colheader" style=""&gt;x0&lt;/th&gt;;  ---10: &lt;th aria-colindex="11" aria-rowindex="1" role="colheader" style=""&gt;y0&lt;/th&gt;;  ---11: &lt;th aria-colindex="12" aria-rowindex="1" role="colheader" style=""&gt;y0&lt;/th&gt;;  ---12: &lt;th aria-colindex="13" aria-rowindex="1" role="colheader" style=""&gt;y0&lt;/th&gt;;  ---13: &lt;th aria-colindex="14" aria-rowindex="1" role="colheader" style=""&gt;y0&lt;/th&gt;;  ---14: &lt;th aria-colindex="15" aria-rowindex="1" role="colheader" style=""&gt;y0&lt;/th&gt;;  ---15: &lt;th aria-colindex="16" aria-rowindex="1" role="colheader" style=""&gt;y0&lt;/th&gt;;  ---16: &lt;th aria-colindex="17" aria-rowindex="1" role="colheader" style=""&gt;y0&lt;/th&gt;;  ---17: &lt;th aria-colindex="18" aria-rowindex="1" aria-sort="ascending" role="colheader" style=""&gt;y0&lt;/th&gt;;  ---18: &lt;th aria-colindex="19" aria-rowindex="1" role="colheader" 

In [9]:
%%
visual styling for the graph

```css
#edges.visual {
    --w: 400;
    --h: 400;
    width: var(--w); height: calc(200px + var(--h) * 1px);
    display: block;
    padding: 100px;
    tbody {
        position: relative;
        display: inline-block;
        width: var(--w); height: var(--h);
        tr {
            display: grid;
            grid-template-columns: 1fr;
            grid-template-rows: 1fr;
            position: absolute;
            --X0: calc((var(--x0) - var(--x0-min)) / var(--x0-diff) * var(--w));
            --Y0: calc((var(--y0) - var(--y0-min)) / var(--y0-diff) * var(--h));
            --X1: calc((var(--x1) - var(--x0-min)) / var(--x0-diff) * var(--w));
            --Y1: calc((var(--y1) - var(--y0-min)) / var(--y0-diff) * var(--h));
            --d: calc(
                pow(pow(var(--X1) - var(--X0), 2) + pow(var(--Y1) - var(--Y0), 2), .5)
            );
            --r: calc(atan2(var(--Y1) - var(--Y0), var(--X1) - var(--X0)));
            th {
                position: absolute;
                display: inline-block;
                transition: 2s all;
                &:nth-of-type(1) {
                    transform:
                        translate(
                            calc(var(--X0) * 1px), calc(var(--Y0) * 1px)
                        )
                        translate(-50%, -50%)
                        ;
                }
                &:nth-of-type(2) {
                    transform:
                        translate(
                            calc(var(--X1) * 1px), calc(var(--Y1) * 1px)
                        )
                        translate(-50%, -50%)
                        
                        ;
                }
            }  
            td {
                display: none;
            }
            &::before {
                        content: "";
                        display: content;
                        position: absolute;
                        transform-origin: calc(1px *var(--X0)) calc(1px *var(--Y0));
                        height: 1px;
                        border: solid 2px;
                        transition: 2s all;
                        width: calc(var(--d) * 1px);
                        transform: 
                            rotate(var(--r))
                            translate(
                                calc(var(--X0) * 1px), calc(var(--Y0) * 1px)
                            )
                            ;
                    }
        }
        
    }
```

'#edges.visual {\n    --w: 400;\n    --h: 400;\n    width: var(--w); height: calc(200px + var(--h) * 1px);\n    display: block;\n    padding: 100px;\n    tbody {\n        position: relative;\n        display: inline-block;\n        width: var(--w); height: var(--h);\n        tr {\n            display: grid;\n            grid-template-columns: 1fr;\n            grid-template-rows: 1fr;\n            position: absolute;\n            --X0: calc((var(--x0) - var(--x0-min)) / var(--x0-diff) * var(--w));\n            --Y0: calc((var(--y0) - var(--y0-min)) / var(--y0-diff) * var(--h));\n            --X1: calc((var(--x1) - var(--x0-min)) / var(--x0-diff) * var(--w));\n            --Y1: calc((var(--y1) - var(--y0-min)) / var(--y0-diff) * var(--h));\n            --d: calc(\n                pow(pow(var(--X1) - var(--X0), 2) + pow(var(--Y1) - var(--Y0), 2), .5)\n            );\n            --r: calc(atan2(var(--Y1) - var(--Y0), var(--X1) - var(--X0)));\n            th {\n                position: absolute;\n                display: inline-block;\n                transition: 2s all;\n                &:nth-of-type(1) {\n                    transform:\n                        translate(\n                            calc(var(--X0) * 1px), calc(var(--Y0) * 1px)\n                        )\n                        translate(-50%, -50%)\n                        ;\n                }\n                &:nth-of-type(2) {\n                    transform:\n                        translate(\n                            calc(var(--X1) * 1px), calc(var(--Y1) * 1px)\n                        )\n                        translate(-50%, -50%)\n                        \n                        ;\n                }\n            }  \n            td {\n                display: none;\n            }\n            &::before {\n                        content: "";\n                        display: content;\n                        position: absolute;\n                        transform-origin: calc(1px *var(--X0)) calc(1px *var(--Y0));\n                        height: 1px;\n                        border: solid 2px;\n                        transition: 2s all;\n                        width: calc(var(--d) * 1px);\n                        transform: \n                            rotate(var(--r))\n                            translate(\n                                calc(var(--X0) * 1px), calc(var(--Y0) * 1px)\n                            )\n                            ;\n                    }\n        }\n        \n    }\n'

## marginalia

what describes the figure and form of a graph?

the graph form has many typographical layouts that new to be considered

there are two tables, two maps, a legend of nodes, and an index of relative positions.

nodes and edges

nodes are primary foreground forms
edges are second mid ground forms
all other attributes are background